# 二维数据结构：DataFrame

In [2]:
import numpy as np
import pandas as pd

`DataFrame` 是 `pandas` 中的二维数据结构，可以看成一个 `Excel` 中的工作表，或者一个 `SQL` 表，或者一个存储 `Series` 对象的字典。

`DataFrame(data, index, columns)` 中的 `data` 可以接受很多数据类型：

- 一个存储一维数组，字典，列表或者 `Series` 的字典
- 2-D 数组
- 结构或者记录数组
- 一个 `Series`
- 另一个 `DataFrame`

`index` 用于指定行的 `label`，`columns` 用于指定列的 `label`，如果参数不传入，那么会按照传入的内容进行设定。

## 从 Series 字典中构造

可以使用值为 `Series` 的字典进行构造：

In [7]:
d = {'one' : pd.Series([1.,2.,3.], index=['a','b','c']),
     'two' : pd.Series([1.,2.,3.,4.], index=['a','b','c','d'])}
d

{'one': a    1.0
 b    2.0
 c    3.0
 dtype: float64,
 'two': a    1.0
 b    2.0
 c    3.0
 d    4.0
 dtype: float64}

如果没有传入 `columns` 的值，那么 `columns` 的值默认为字典 `key`，`index` 默认为所有 `value` 中 `index` 的并集。

In [8]:
df = pd.DataFrame(d)
df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


如果指定了index值，index为指定的index值

In [9]:
pd.DataFrame(d, index=['d', 'b', 'a'])

,one,two
d,NaN,4.0
b,2.0,2.0
a,1.0,1.0


如果指定了columns值，会去字典中寻找，找不到的值为NaN

In [10]:
pd.DataFrame(d, index=['d','b','a'], columns=['two','three'])

,two,three
d,4.0,NaN
b,2.0,NaN
a,1.0,NaN


查看index和columns

In [11]:
df.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [12]:
df.columns

Index(['one', 'two'], dtype='object')

## 从ndarray或者list字典中构造

如果字典是ndarray或者list，那么他们的长度要严格保持一致

In [13]:
d = {'one': [1.,2.,3.,4.],
     'two': [4.,3.,2.,1.]}

index默认为range(n)，其中n为数组长度

In [14]:
pd.DataFrame(d)

,one,two
0,1.0,4.0
1,2.0,3.0
2,3.0,2.0
3,4.0,1.0


如果传入index参数，那么他必须与数组等长

In [15]:
pd.DataFrame(d, index=['a','b','c','d'])

,one,two
a,1.0,4.0
b,2.0,3.0
c,3.0,2.0
d,4.0,1.0


## 从结构数组中构造

numpy支持结构数组的构造

In [19]:
data = np.zeros((2,), dtype=[('A', 'i4'), ('B', 'f4'), ('C', 'a10')])
data[:] = [(1,2.,"Hello"), (2,3.,'World')]
data

array([(1, 2., b'Hello'), (2, 3., b'World')],
      dtype=[('A', '<i4'), ('B', '<f4'), ('C', 'S10')])

参数处理的方式与数组字典类似

In [20]:
pd.DataFrame(data)

,A,B,C
0,1,2.0,b'Hello'
1,2,3.0,b'World'


In [21]:
pd.DataFrame(data, index=['first', 'second'])

,A,B,C
first,1,2.0,b'Hello'
second,2,3.0,b'World'


In [22]:
pd.DataFrame(data, columns=['C', 'A', 'B'])

,C,A,B
0,b'Hello',1,2.0
1,b'World',2,3.0


## 从字典列表中构造

字典中同一键的值会被合并到同一列

In [23]:
data2 = [{'a':1, 'b':2}, {'a':5, 'b':10, 'c':20}]
pd.DataFrame(data2)

,a,b,c
0,1,2,NaN
1,5,10,20.0


In [24]:
pd.DataFrame(data2, index=['first', 'second'])

,a,b,c
first,1,2,NaN
second,5,10,20.0


In [25]:
pd.DataFrame(data2, columns=['a','b'])

,a,b
0,1,2
1,5,10


## 从Series中构造

相当于将Series二维化

## 其他构造方法

`DataFrame.from_dict` 从现有的一个字典中构造，`DataFrame.from_records` 从现有的一个记录数组中构造：

In [26]:
pd.DataFrame.from_records(data, index='C')

,A,B
C,,
b'Hello',1,2.0
b'World',2,3.0


`DataFrame.from_items` 从字典的 `item` 对构造：

In [29]:
pd.DataFrame.from_dict([('A',[1,2,3]), ('B', [4,5,6])])

,0,1
0,A,"[1, 2, 3]"
1,B,"[4, 5, 6]"


## 列操作

DataFrame可以类似于字典一样对列进行操作

In [30]:
df['one']

a    1.0
b    2.0
c    3.0
d    NaN
Name: one, dtype: float64

添加新列

In [32]:
df['three'] = df['one'] * df['two']
df['flag'] = df['one'] > 2
df

,one,two,three,flag
a,1.0,1.0,1.0,False
b,2.0,2.0,4.0,False
c,3.0,3.0,9.0,True
d,NaN,4.0,NaN,False


可以像字典一样删除

In [33]:
del df['two']
three = df.pop('three')
df

,one,flag
a,1.0,False
b,2.0,False
c,3.0,True
d,NaN,False


给一行赋单一值

In [34]:
df['foo'] = 'bar'
df

,one,flag,foo
a,1.0,False,bar
b,2.0,False,bar
c,3.0,True,bar
d,NaN,False,bar


如果 `index` 不一致，那么会只保留公共的部分：

In [36]:
df['one_trunc'] = df['one'][:2]
df

,one,flag,foo,one_trunc
a,1.0,False,bar,1.0
b,2.0,False,bar,2.0
c,3.0,True,bar,NaN
d,NaN,False,bar,NaN


也可以直接插入一维数组，但是数组的长度必须与 `index` 一致。

默认新列插入位置在最后，也可以指定位置插入：

In [39]:
df.insert(1, 'some', df['one'])
df

,one,some,bar,flag,foo,one_trunc
a,1.0,1.0,1.0,False,bar,1.0
b,2.0,2.0,2.0,False,bar,2.0
c,3.0,3.0,3.0,True,bar,NaN
d,NaN,NaN,NaN,False,bar,NaN


添加一个test新列

In [40]:
df.assign(test=df['one'] + df['bar'])

,one,some,bar,flag,foo,one_trunc,test
a,1.0,1.0,1.0,False,bar,1.0,2.0
b,2.0,2.0,2.0,False,bar,2.0,4.0
c,3.0,3.0,3.0,True,bar,NaN,6.0
d,NaN,NaN,NaN,False,bar,NaN,NaN


## 索引和选择

基本操作：

| Operation	| Syntax | Result |
| ---- | ----- | ---- |
| Select column	| df[col] | Series |
| Select row by label | df.loc[label] | Series |
| Select row by integer location | df.iloc[loc] | Series |
| Slice rows | df[5:10]	| DataFrame |
| Select rows by boolean vector	| df[bool_vec] | DataFrame |